In [1]:
import numpy as np
import function as F

# 初期条件の設定

In [2]:
D=4
adj=np.array([[0,1,0,0],[1,0,1,1],[0,1,0,1],[0,1,1,0]])
x_0=np.zeros((D,len(adj)))
x_0[0]=1
W=np.array([[2,0,0,0],[-1,0,0,0],[0,1,0,0],[1,0,0,0]])
A=np.array([1,1,1,1])
b=np.array([1])

In [3]:
x_0

array([[1., 1., 1., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

#### 集約-1

In [4]:
a_0=np.zeros_like(x_0.T)

for i in range(len(adj)):
    a_0[i]=x_0.T[adj[i]==1].sum(axis=0)
    
print(a_0.T)

[[1. 3. 2. 2.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


#### 集約-2
function.py内でrelu関数を定義している

In [5]:
x_1=F.relu(W@a_0.T)
print(x_1)

[[2. 6. 4. 4.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 3. 2. 2.]]


### ２回目の集約

In [6]:
a_1=np.zeros_like(x_1.T)

for i in range(len(adj)):
    a_1[i]=x_1.T[adj[i]==1].sum(axis=0)
    
print(a_1.T)

[[ 6. 10. 10. 10.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 3.  5.  5.  5.]]


In [7]:
x_2=F.relu(W@a_1.T)
print(x_2)

[[12. 20. 20. 20.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 6. 10. 10. 10.]]


### h_Gを求める

In [8]:
h_G=np.sum(x_2, axis=1)

In [9]:
print(h_G)

[72.  0.  0. 36.]


### 重み付き和の算出

In [10]:
affine=A@h_G+b

In [11]:
print(affine)

[109.]


## class GNNモデルを作成する

model.pyにGNNクラスを作成した。
以下のようなattributeを作成した。

|attribute||
|---|---|
|aggregate|隣接行列を引数にとり集約された結果を返す initilize=Trueで設定してあるx_vecsを初期化|
|get_h_G|隣接行列とTをとり、T(defaultで2)回集約したあとh_Gの特徴ベクトルを返す、initilize=Trueで設定してあるx_vecsを初期化|
|affine|隣接行列をとり、重み付き和を返す|

In [12]:
import model
gnn=model.GNN(D)

In [13]:
gnn.theta["W"]=W
gnn.theta["A"]=A
gnn.theta["b"]=b
gnn.x_vecs=x_0

In [14]:
gnn.aggregate(adj, initilize=False)

array([[2., 6., 4., 4.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 3., 2., 2.]])

In [15]:
gnn.get_h_G(adj, T=2, initilize=False) #T=2の集約の結果。上で計算しているものと一致している。

array([72.,  0.,  0., 36.])

In [17]:
gnn.aggregate(adj) #initilize=Trueでrandom.seedが固定された正規分布に従ったx_vecsに初期化される

array([[0.        , 3.94300006, 0.        , 1.44905743],
       [0.4934171 , 0.        , 0.05664633, 0.        ],
       [0.43030744, 0.        , 0.        , 0.        ],
       [0.        , 1.97150003, 0.        , 0.72452871]])

In [18]:
gnn.x_vecs

array([[ 0.31678346, -0.4934171 ,  1.21794581,  0.43677076],
       [-0.72769267,  0.43030744, -0.68666532, -0.45301801],
       [-0.73285921, -0.75362743, -0.83251892,  0.44987187],
       [ 1.80752304, -0.48953136,  0.88158874, -0.69834087]])